In [110]:
import pandas as pd
import pickle
import re
from  sklearn.ensemble import RandomForestClassifier
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import resample
from datetime import *
from statistics import mean
from random import seed
from random import randint

In [111]:
user_decision = int(input('Enter 1 to read Chicago dataset and enter 2 to read Las-Vegas dataset'))

Enter 1 to read Chicago dataset and enter 2 to read Las-Vegas dataset2


In [112]:
if(user_decision==1):
    df = pd.read_csv('chicago_data_frame_with_fewer_rows.csv')
    df = df.set_index(['hotel_id', 'reviewer_id'])
    with open("reviews_file_chicago.txt", "rb") as fp:   # Unpickling
        review_count = pickle.load(fp)
elif(user_decision ==2):
    df = pd.read_csv('las-vegas_data_frame_with_fewer_rows.csv')
    df = df.set_index(['hotel_id', 'reviewer_id'])
    with open("reviews_file_las-vegas.txt", "rb") as fp:   # Unpickling
        review_count = pickle.load(fp)

In [113]:
regex = r'<hotelUrl>\n(.*?)\n</hotelUrl>'
unique_hotel_ids = set()
for i in review_count:
    hotel_name = re.search(regex, i)
    hotel_name = hotel_name[1]
    unique_hotel_ids.add(hotel_name)

In [114]:
def convert_to_date(date):
    ans = datetime.strptime(date, "%b %d, %Y")
    return ans

In [115]:
regex = r'<hotelUrl>\n(.*?)\n</hotelUrl>\n[\s\S]+?\n<helpfulness>\n(.*?)\n</helpfulness>'
unique_hotel_ids = set()
for i in review_count:
    hotel_name = re.search(regex, i)
    hotel_id = hotel_name[1]
    helpful_score = (hotel_name[2]).split(' of ')
    if (int(helpful_score[1])) > 4:
        unique_hotel_ids.add(hotel_id)


In [116]:
date_member_hotel = []
obj = []
count = 0
date_member_hotel_regex = r'<memberUrl>\n([\s\S]*?)\n</memberUrl>\n<hotelUrl>\n([\s\S]*?)\n</hotelUrl>\n[\s\S]*?\n<date>\n(.*?)\n</date>'
for i in review_count:
    matched = re.search(date_member_hotel_regex, i)
    member = matched[1]
    hotel = matched[2]
    date = matched[3]
    obj.append([hotel, member, date])

In [117]:
rows_with_hotel_as_key = dict()

for i in unique_hotel_ids:
    if i in df.index.get_level_values('hotel_id'):
        if i in rows_with_hotel_as_key:
            rows_with_hotel_as_key[i] = pd.merge(rows_with_hotel_as_key[i], df.loc[[i]], how = 'right', on=['hotel_id', 'reviewer_id'])
        else:
            rows_with_hotel_as_key[i] = pd.DataFrame(df.loc[[i]])

In [118]:
def removekey(d, key):
    r = dict(d)
    r.pop(key)
    return r

In [119]:
rows_with_hotel_as_key_culled_neg = dict()

for i in rows_with_hotel_as_key:
    if len(rows_with_hotel_as_key[i]) < 5:
        continue
    else:
        neg = 0
        for j,row in rows_with_hotel_as_key[i].iterrows():
            if (row['ST1']) < 4:
                neg+=1
        if neg > 4:
            rows_with_hotel_as_key_culled_neg[i] = rows_with_hotel_as_key[i]

In [120]:
len(rows_with_hotel_as_key_culled_neg)

106

In [121]:
rows_with_hotel_as_key_culled_pos = dict()

for i in rows_with_hotel_as_key:
    if len(rows_with_hotel_as_key[i]) < 5:
        continue
    else:
        pos = 0
        for j,row in rows_with_hotel_as_key[i].iterrows():
            if (row['ST1']) >= 4:
                pos+=1
        if pos > 4:
            rows_with_hotel_as_key_culled_pos[i] = rows_with_hotel_as_key[i]

In [122]:
len(rows_with_hotel_as_key_culled_pos)

267

In [123]:
x_train = pd.DataFrame(columns=df.columns)
x_train = pd.DataFrame(columns=df.columns)
x_train['hotel_id'] = ''
x_train['reviewer_id'] = ''
x_train = x_train.set_index(['hotel_id', 'reviewer_id'])

In [124]:
classifier = RandomForestClassifier()
hotel_prediction = dict()
y_pred = []
y_test = []
y_pred_df_neg = dict()
for i in rows_with_hotel_as_key_culled_neg:
    #create a dataframe I can use to train using leave one out approach
    # i is the hotel we want to use to test the classifier which is trained using the other hotels
    x_test = rows_with_hotel_as_key_culled_neg[i].copy()
    y_train = []
    x_train = pd.DataFrame(columns=df.columns)
    for j in rows_with_hotel_as_key_culled_neg:
        if i != j:
            new_df = rows_with_hotel_as_key_culled_neg[j].copy()
            x_train = pd.concat([x_train, new_df])
    y_train = x_train.pop('helpful_of_not')
    y_train = (y_train.to_frame()['helpful_of_not']).astype(str).astype(int)
    classifier.fit(X=x_train.values, y=y_train.values)
    y_test.append(x_test.pop('helpful_of_not'))
    prediction = classifier.predict_proba(x_test)
    y_pred_df_neg[x_test.index[0][0]] = pd.DataFrame(prediction, columns = ['not helpful', 'helpfulness'], index=x_test.index)


/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [125]:
for i in y_pred_df_neg:
    y_pred_df_neg[i] = pd.merge(y_pred_df_neg[i], df, how = 'left', on=['hotel_id', 'reviewer_id'])

In [126]:
y_pred_fin_neg = dict()

for i in (y_pred_df_neg):
    count =0
    for j,row in y_pred_df_neg[i].iterrows():
        y_pred_fin_neg[i] = y_pred_df_neg[i][y_pred_df_neg[i]['ST1'] < 4]

In [127]:
for i in (y_pred_fin_neg):
    y_pred_fin_neg[i].sort_values(by=['helpfulness'], inplace=True, ascending=False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [128]:
neg_class_avg = []

for i in (y_pred_fin_neg):
    top = y_pred_fin_neg[i].iloc[0]
    neg_class_avg.append((top['helpfulness']))

In [129]:
neg_class_total = []

for i in (y_pred_fin_neg):
    top = y_pred_fin_neg[i].iloc[0]
    helpful = (top['helpfulness'])
    if helpful >= 0.5:
        neg_class_total.append(1)
    else:
        neg_class_total.append(0)

In [130]:
mean(neg_class_avg)

0.8886792452830189

In [131]:
mean(neg_class_total)

1

In [132]:
x_train_pos = pd.DataFrame(columns=df.columns)
x_train_pos = pd.DataFrame(columns=df.columns)
x_train_pos['hotel_id'] = ''
x_train_pos['reviewer_id'] = ''
x_train_pos = x_train_pos.set_index(['hotel_id', 'reviewer_id'])

In [133]:
classifier = RandomForestClassifier(n_estimators=39, max_depth=20)
hotel_prediction = dict()
y_pred = []
y_test = []
y_pred_df_pos = dict()
for i in rows_with_hotel_as_key_culled_pos:
    #create a dataframe I can use to train using leave one out approach
    # i is the hotel we want to use to test the classifier which is trained using the other hotels
    x_test = rows_with_hotel_as_key_culled_pos[i].copy()
    y_train = []
    x_train = pd.DataFrame(columns=df.columns)
    for j in rows_with_hotel_as_key_culled_pos:
        if i != j:
            new_df = rows_with_hotel_as_key_culled_pos[j].copy()
            x_train_pos = pd.concat([x_train, new_df])
    y_train_pos = x_train_pos.pop('helpful_of_not')
    y_train = (y_train_pos.to_frame()['helpful_of_not']).astype(str).astype(int)
    classifier.fit(X=x_train_pos.values, y=y_train.values)
    y_test.append(x_test.pop('helpful_of_not'))
    prediction = classifier.predict_proba(x_test)
    y_pred_df_pos[x_test.index[0][0]] = pd.DataFrame(prediction, columns = ['not helpful', 'helpfulness'], index=x_test.index)
    

In [134]:
for i in y_pred_df_pos:
    y_pred_df_pos[i] = pd.merge(y_pred_df_pos[i], df, how = 'left', on=['hotel_id', 'reviewer_id'])

In [135]:
y_pred_fin_pos = dict()

for i in (y_pred_df_pos):
    count =0
    for j,row in y_pred_df_pos[i].iterrows():
        y_pred_fin_pos[i] = y_pred_df_pos[i][y_pred_df_pos[i]['ST1'] >= 4]

In [136]:
for i in (y_pred_fin_pos):
    y_pred_fin_pos[i].sort_values(by=['helpfulness'], inplace=True, ascending=False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [137]:
pos_class_avg = []

for i in (y_pred_fin_pos):
    top = y_pred_fin_pos[i].iloc[0]
    pos_class_avg.append((top['helpfulness']))

In [138]:
pos_class_total = []

for i in (y_pred_fin_pos):
    top = y_pred_fin_pos[i].iloc[0]
    helpful = (top['helpfulness'])
    if helpful >= 0.5:
        pos_class_total.append(1)
    else:
        pos_class_total.append(0)

In [139]:
mean(pos_class_avg)

0.6406415058100451

In [140]:
mean(pos_class_total)

0.8539325842696629

# Date Negative Mean Review Helpfulness

In [141]:
for i in y_pred_fin_neg:
    y_pred_fin_neg[i]['date'] = ''

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [142]:
for j in y_pred_fin_neg:
    for i in obj: 
        test_tuple = tuple([i[0],i[1]])
        if test_tuple in y_pred_fin_neg[j].index:
            date = convert_to_date(i[2])
            y_pred_fin_neg[j].set_value(y_pred_fin_neg[j].loc[[test_tuple]].index,'date',  date)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/anaconda3/lib/python3.7/site-packages/ipykernel/zmqshell.py:536: PerformanceWarning: indexing past lexsort depth may impact performance.
  return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)


In [143]:
for i in (y_pred_fin_neg):
    y_pred_fin_neg[i].sort_values(by=['date'], inplace=True, ascending=False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [144]:
hold = y_pred_fin_neg.copy()
for i in hold:
    if len(hold[i]) < 1:
        del y_pred_fin_neg[i]

In [145]:
values_helpful_of_not_neg = df['helpful_of_not']

In [146]:
for i in y_pred_fin_neg:
    y_pred_fin_neg[i] = pd.merge(y_pred_fin_neg[i], values_helpful_of_not_neg, how = 'left', on=['hotel_id', 'reviewer_id'])

In [147]:
for i in y_pred_fin_neg:
    y_pred_fin_neg[i] = pd.merge(y_pred_fin_neg[i], values_helpful_of_not_neg, how = 'left', on=['hotel_id', 'reviewer_id'])

In [148]:
for i in (y_pred_fin_neg):
    y_pred_fin_neg[i].sort_values(by=['date'], inplace=True, ascending=False)

In [149]:
total = []
for i in (y_pred_fin_neg):
    top = (y_pred_fin_neg[i].iloc[0])['helpful_of_not']
    total.append(top)

total_fin = list(map(int, total))

In [150]:
mean(total_fin)

0.36792452830188677

# Positive Date Mean Review Helpfulness

In [151]:
for i in y_pred_fin_pos:
    y_pred_fin_pos[i]['date'] = ''

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [152]:
for j in y_pred_fin_pos:
    for i in obj: 
        test_tuple = tuple([i[0],i[1]])
        if test_tuple in y_pred_fin_pos[j].index:
            date = convert_to_date(i[2])
            y_pred_fin_pos[j].set_value(y_pred_fin_pos[j].loc[[test_tuple]].index,'date',  date)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [153]:
for i in (y_pred_fin_pos):
    y_pred_fin_pos[i].sort_values(by=['date'], inplace=True, ascending=False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [154]:
hold = y_pred_fin_pos.copy()
for i in hold:
    if len(hold[i]) < 1:
        del y_pred_fin_pos[i]

In [155]:
values_helpful_of_not_pos = df['helpful_of_not']

In [156]:
for i in y_pred_fin_pos:
    y_pred_fin_pos[i] = pd.merge(y_pred_fin_pos[i], values_helpful_of_not_pos, how = 'left', on=['hotel_id', 'reviewer_id'])

In [157]:
for i in y_pred_fin_pos:
    y_pred_fin_pos[i] = pd.merge(y_pred_fin_pos[i], values_helpful_of_not_pos, how = 'left', on=['hotel_id', 'reviewer_id'])

In [158]:
for i in (y_pred_fin_pos):
    y_pred_fin_pos[i].sort_values(by=['date'], inplace=True, ascending=False)

In [159]:
total = []
for i in (y_pred_fin_pos):
    top = (y_pred_fin_pos[i].iloc[0])['helpful_of_not']
    total.append(top)

total_fin = list(map(int, total))

In [160]:
mean(total_fin)

0.5056179775280899

# Random Negative Percentage of recommended reviews that are helpful

In [161]:
for i in y_pred_fin_neg:
    del y_pred_fin_neg[i]['date']

In [162]:
for i in y_pred_fin_neg:
    rand_int = randint(0,len(y_pred_fin_neg[i])-1)
    top = (y_pred_fin_neg[i].iloc[rand_int])['helpful_of_not']
    total.append(top)

In [163]:
total_fin_neg = list(map(int, total))

In [164]:
mean(total_fin_neg)

0.4718498659517426

In [165]:
df.pop('helpful_of_not')

hotel_id                                                                                                                                           reviewer_id                                              
http://las-vegas-hotels.tripadvisor.com/Hotel_Review-g45963-d91948-Reviews-Texas_Station_Gambling_Hall_and_Hotel-Las_Vegas_Nevada.html             /MemberProfile-ccu-a_uid.FF7879ECEFFD88884C5302F1B7F11867    0
http://las-vegas-hotels.tripadvisor.com/Hotel_Review-g45963-d97733-Reviews-Sahara_Hotel_Casino-Las_Vegas_Nevada.html                               /MemberProfile-ccu-a_uid.CA0357067C1B45B2A93075B7EBE65995    0
/Hotel_Review-g31377-d74308-Reviews-Embassy_Suites_Hotel_Phoenix_Tempe-Tempe_Arizona.html                                                          /members-reviews/JulieMCCoy                                  0
http://las-vegas-hotels.tripadvisor.com/Hotel_Review-g45963-d114892-Reviews-The_Westin_Casuarina_Las_Vegas_Hotel_Casino_Spa-Las_Vegas_Nevada.html  /members-reviews/m

In [166]:
if user_decision==1:
    vals_df = pd.read_csv('vals_df.csv')
    vals_df = vals_df.set_index(['hotel_id', 'reviewer_id'])
    df = pd.merge(vals_df, df, how = 'left', on=['hotel_id', 'reviewer_id'])
else:
    vals_df = pd.read_csv('las-vegas_vals_df.csv')
    vals_df = vals_df.set_index(['hotel_id', 'reviewer_id'])
    df = pd.merge(vals_df, df, how = 'left', on=['hotel_id', 'reviewer_id'])

In [167]:
rows_with_hotel_as_key = dict()

for i in unique_hotel_ids:
    if i in df.index.get_level_values('hotel_id'):
        if i in rows_with_hotel_as_key:
            rows_with_hotel_as_key[i] = pd.merge(rows_with_hotel_as_key[i], df.loc[[i]], how = 'right', on=['hotel_id', 'reviewer_id'])
        else:
            rows_with_hotel_as_key[i] = pd.DataFrame(df.loc[[i]])

In [168]:
rows_with_hotel_as_key_culled_neg = dict()

for i in rows_with_hotel_as_key:
    if len(rows_with_hotel_as_key[i]) < 5:
        continue
    else:
        neg = 0
        for j,row in rows_with_hotel_as_key[i].iterrows():
            if (row['ST1']) < 4:
                neg+=1
        if neg > 4:
            rows_with_hotel_as_key_culled_neg[i] = rows_with_hotel_as_key[i]

In [169]:
for i in y_pred_fin_neg:
    y_pred_fin_neg[i]['date'] = ''

In [170]:
for j in y_pred_fin_neg:
    for i in obj: 
        test_tuple = tuple([i[0],i[1]])
        if test_tuple in y_pred_fin_neg[j].index:
            date = convert_to_date(i[2])
            y_pred_fin_neg[j].set_value(y_pred_fin_neg[j].loc[[test_tuple]].index,'date',  date)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [171]:
for i in (y_pred_fin_neg):
    y_pred_fin_neg[i].sort_values(by=['date'], inplace=True, ascending=False)

In [172]:
total = []
for i in (y_pred_fin_neg):
    top = (y_pred_fin_neg[i].iloc[0])['helpfulness']
    total.append(top)

In [173]:
print(type(total))
total_fin = list(map(float, total))

<class 'list'>


In [174]:
mean(total_fin)

0.3877358490566038

# Random Positive Percentage of recommended reviews that are helpful

In [175]:
for i in y_pred_fin_pos:
    rand_int = randint(0,len(y_pred_fin_pos[i])-1)
    top = (y_pred_fin_pos[i].iloc[rand_int])['helpful_of_not']
    total.append(top)

In [176]:
total_fin_pos = list(map(int, total))

In [177]:
mean(total_fin_pos)

0.4584450402144772

In [178]:
df.pop('helpful_of_not')

hotel_id                                                                                                                  reviewer_id                
/Hotel_Review-g34352-d267184-Reviews-Hapimag_Orlando_Lake_Berkley_Resort-Kissimmee_Florida.html                           /members-reviews/Bathonion     0.833333
/Hotel_Review-g34381-d251992-Reviews-Courtyard_Villa-Lauderdale_by_the_Sea_Florida.html                                   /members-reviews/Bathonion     0.000000
/Hotel_Review-g60971-d75521-Reviews-La_Quinta_Inn_and_Suites_Flagstaff-Flagstaff_Arizona.html                             /members-reviews/Bathonion     1.000000
/Hotel_Review-g61000-d119523-Reviews-Yosemite_Lodge_At_The_Falls-Yosemite_National_Park_California.html                   /members-reviews/Bathonion     1.000000
http://las-vegas-hotels.tripadvisor.com/Hotel_Review-g45963-d121312-Reviews-Bally_s_Las_Vegas-Las_Vegas_Nevada.html       /members-reviews/Bathonion     0.962963
                                        

In [186]:
if user_decision==1:
    vals_df = pd.read_csv('vals_df.csv')
    vals_df = vals_df.set_index(['hotel_id', 'reviewer_id'])
    df = pd.merge(vals_df, df, how = 'left', on=['hotel_id', 'reviewer_id'])
else:
    vals_df = pd.read_csv('las-vegas_vals_df.csv')
    vals_df = vals_df.set_index(['hotel_id', 'reviewer_id'])
    df = pd.merge(vals_df, df, how = 'left', on=['hotel_id', 'reviewer_id'])

In [187]:
rows_with_hotel_as_key = dict()

for i in unique_hotel_ids:
    if i in df.index.get_level_values('hotel_id'):
        if i in rows_with_hotel_as_key:
            rows_with_hotel_as_key[i] = pd.merge(rows_with_hotel_as_key[i], df.loc[[i]], how = 'right', on=['hotel_id', 'reviewer_id'])
        else:
            rows_with_hotel_as_key[i] = pd.DataFrame(df.loc[[i]])

In [188]:
rows_with_hotel_as_key_culled_pos = dict()

for i in rows_with_hotel_as_key:
    if len(rows_with_hotel_as_key[i]) < 5:
        continue
    else:
        pos = 0
        for j,row in rows_with_hotel_as_key[i].iterrows():
            if (row['ST1']) >= 4:
                pos+=1
        if pos > 4:
            rows_with_hotel_as_key_culled_pos[i] = rows_with_hotel_as_key[i]

In [189]:
for i in y_pred_fin_pos:
    y_pred_fin_pos[i]['date'] = ''

In [190]:
for j in y_pred_fin_pos:
    for i in obj: 
        test_tuple = tuple([i[0],i[1]])
        if test_tuple in y_pred_fin_pos[j].index:
            date = convert_to_date(i[2])
            y_pred_fin_pos[j].set_value(y_pred_fin_pos[j].loc[[test_tuple]].index,'date',  date)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [191]:
for i in (y_pred_fin_pos):
    y_pred_fin_pos[i].sort_values(by=['date'], inplace=True, ascending=False)

In [192]:
total = []
for i in (y_pred_fin_pos):
    top = (y_pred_fin_pos[i].iloc[0])['helpfulness']
    total.append(top)

In [193]:
print(type(total))
total_fin = list(map(float, total))

<class 'list'>


In [194]:
mean(total_fin)

0.37683664649956783